# Imports

In [1]:
import os

import tensorflow as tf
import numpy as np

In [2]:
FILE_PATH = os.getcwd()
MODEL_PATH = os.path.join(FILE_PATH, "../models/my_vgg.h5")
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# Datasets

In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def __prepare_datasets():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return (x_train, y_train), (x_test[:20], y_test[:20])

In [4]:
data_train, data_test = __prepare_datasets()
x_train, y_train = data_train
x_test, y_test = data_test
print(f"x_train.shape = {x_train.shape} y_train.shape = {y_train.shape}")
print(f"x_test.shape = {x_test.shape} y_test.shape = {y_test.shape}")

x_train.shape = (50000, 32, 32, 3) y_train.shape = (50000, 10)
x_test.shape = (20, 32, 32, 3) y_test.shape = (20, 10)


# Model

In [5]:
from integration_tests.models.my_vgg import my_vgg

def __prepare_model(data_train, data_test):
    if os.path.exists(MODEL_PATH):
        print("---Using Existing Model---")
        model: tf.keras.Model = tf.keras.models.load_model(MODEL_PATH)
    else:
        print("---Training Model---")
        print(f"GPU IS AVAILABLE: {tf.config.list_physical_devices('GPU')}")
        model: tf.keras.Model = my_vgg()
        model.fit(
            *data_train,
            epochs=100,
            batch_size=64,
            validation_data=data_test,
        )
        model.save(MODEL_PATH)

    model.summary()
    return model

In [6]:
model = __prepare_model(data_train, data_test)

---Using Existing Model---
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            

# Perturbations

In [7]:
def original_perturb_image(xs, img):
    # If this function is passed just one perturbation vector,
    # pack it in a list to keep the computation the same
    if xs.ndim < 2:
        xs = np.array([xs])

    # Copy the image n == len(xs) times so that we can
    # create n new perturbed images
    tile = [len(xs)] + [1] * (xs.ndim + 1)
    imgs = np.tile(img, tile)

    # Make sure to floor the members of xs as int types
    xs = xs.astype(int)

    for x, img in zip(xs, imgs):
        # Split x into an array of 5-tuples (perturbation pixels)
        # i.e., [[x,y,r,g,b], ...]
        pixels = np.split(x, len(x) // 5)
        for pixel in pixels:
            # At each pixel's x,y position, assign its rgb value
            x_pos, y_pos, *rgb = pixel
            img[x_pos, y_pos] = rgb

    return imgs

In [8]:
def predict_classes(
    xs: np.ndarray, img: np.ndarray, y_true: int, model: tf.keras.Model
) -> np.ndarray:
    """Perturb the image and get the predictions of the model."""
    imgs_perturbed = original_perturb_image(xs, img)
    predictions = model.predict(imgs_perturbed)[:, y_true]
    return predictions

In [9]:
from typing import Optional
import logging

def attack_success(
    x: np.ndarray, img: np.ndarray, y_true: int, model: tf.keras.Model
) -> Optional[bool]:
    """Predict ONE image and return True if expected. None otherwise."""
    attack_image = original_perturb_image(x, img)

    confidence = model.predict(attack_image)[0]
    predicted_class = np.argmax(confidence)

    # If the prediction is what we want (misclassification or
    # targeted classification), return True
    logging.debug(f"Confidence: {confidence[y_true]}")
    if predicted_class == y_true:
        return True

In [10]:
from textwrap import dedent
from inputtensorfi.attacks.differential_evolution import differential_evolution

def build_attack(
    model: tf.keras.Model,
    pixel_count=1,
    maxiter=75,
    popsize=400,
    verbose=False,
):
    def attack(
        img: np.ndarray,
        y_true: int,
    ):
        # Define bounds for a flat vector of x,y,r,g,b values
        # For more pixels, repeat this layout
        bounds = [(0, 32), (0, 32), (0, 256), (0, 256), (0, 256)] * pixel_count

        # Population multiplier, in terms of the size of the perturbation vector x
        popmul = max(1, popsize // len(bounds))

        # Format the predict/callback functions for the differential evolution algorithm
        def predict_fn(xs):
            return predict_classes(xs, img, y_true, model)

        def callback_fn(x, convergence):
            return attack_success(
                x,
                img,
                y_true,
                model,
            )

        # Call Scipy's Implementation of Differential Evolution
        attack_result = differential_evolution(
            predict_fn,
            bounds,
            maxiter=maxiter,
            popsize=popmul,
            recombination=1,
            atol=-1,
            callback=callback_fn,
            polish=False,
        )

        if verbose:
            # Calculate some useful statistics to return from this function
            attack_image = original_perturb_image(attack_result.x, img)[0]
            prior_probs = model.predict(np.array([img]))[0]
            prior_class = np.argmax(prior_probs)
            predicted_probs = model.predict(np.array([attack_image]))[0]
            predicted_class = np.argmax(predicted_probs)
            success = predicted_class != y_true
            cdiff = prior_probs[y_true] - predicted_probs[y_true]

            print(
                dedent(
                    "-- TRUTH --\n"
                    f"y_true={y_true}\n"
                    "-- W/O FI PREDS --\n"
                    f"prior_probs={prior_probs}\n"
                    f"prior_class={prior_class}\n"
                    "-- FI PREDS --\n"
                    f"attack_results={attack_result.x}\n"
                    f"predicted_probs={predicted_probs}\n"
                    f"predicted_class={predicted_class}\n"
                    f"success={success}\n"
                    f"cdiff={cdiff}\n"
                )
            )

        return attack_result.x

    return attack

Convert the fault injector based on numpy to a tensor.

In [11]:
def build_attack_as_tf(
    model: tf.keras.Model,
    pixel_count=1,
    maxiter=75,
    popsize=400,
    verbose=False,
):
    attack = build_attack(
        model,
        pixel_count=pixel_count,
        maxiter=maxiter,
        popsize=popsize,
        verbose=verbose,
    )

    def attack_as_tf(img, y_true):
        return tf.numpy_function(attack, [img, y_true], tf.double)

    return attack_as_tf

Create a new tensor to perform the parallel calculation.

In [12]:
def build_parallel_attack_as_tf(
    model,
    pixel_count=1,
    maxiter=75,
    popsize=400,
    verbose=False,
):
    @tf.function
    def parallel_attack_as_tf(
        imgs,
        y_trues,
    ):
        attack_as_tf = build_attack_as_tf(
            model,
            pixel_count=pixel_count,
            maxiter=maxiter,
            popsize=popsize,
            verbose=verbose,
        )
        return tf.vectorized_map(
            lambda x: attack_as_tf(x[0], x[1]), elems=[imgs, y_trues]
        )

    return parallel_attack_as_tf


# Test

Avoid working in categorical (binary matrix classes).

In [13]:
y_true = np.array([np.argmax(y) for y in y_test])
y_true

array([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6],
      dtype=int64)

In [14]:
build_parallel_attack_as_tf(model)(x_test, y_true).numpy()

array([[1.33434186e+01, 2.18139778e+01, 2.95123994e+01, 4.75965967e+01,
        5.92697516e+01],
       [2.65945576e+01, 8.64797740e+00, 2.22553328e+02, 2.05020628e+02,
        1.72759285e+02],
       [2.39763153e+01, 2.33670395e+01, 2.55391322e+02, 2.55775207e+02,
        2.15465893e+02],
       [2.27173569e+01, 4.28794542e+00, 6.63671927e-02, 4.82242720e-01,
        2.07104240e-01],
       [1.52169177e+01, 2.36333808e+01, 1.89231426e+01, 2.00517746e+01,
        2.14268126e+01],
       [2.40617401e+01, 2.01067137e+01, 2.21243086e+02, 1.60033769e+02,
        2.33612946e+02],
       [2.96925734e+01, 1.16100369e+01, 2.54726614e+02, 2.55755097e+02,
        2.55202340e+02],
       [1.44771991e+01, 1.57277695e+01, 2.55752698e+02, 2.40635936e+02,
        1.83080361e+02],
       [8.82964159e+00, 1.04277558e+01, 2.70171034e+01, 4.24620191e-01,
        2.18372455e-01],
       [1.90627662e+00, 1.87983121e+01, 2.31881191e+01, 7.06750622e+01,
        5.76902802e+01],
       [2.63527472e+01, 2.5216